In [1]:
pip install nltk scikit-learn streamlit


     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 505.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.7 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.5 MB 4.0 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.5 MB 4.7 MB/s eta 0:00:01
   ----------------- ---------------------- 0.6/1.5 MB 5.1 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.5 MB 4.8 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Importing necessary libraries
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [16]:
intents = [
    {
        'tag':'greeting',
        'patterns':['Hi', 'Hello', 'Hey', 'Whats up', 'How are you'],
        'responses':['Hi there', 'Hello', 'Hey', 'Nothing much', 'Im fine, thank you']
    },
    {
        'tag':'goodbye',
        'patterns':['Bye', 'See you later', 'Goodbye', 'Take care'],
        'responses':['Goodbye', 'See you later', 'Take care']
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
        "tag": "jokes",
        "patterns": ["Tell me a joke", "Make me laugh", "Do you know any jokes?"],
        "responses": ["Why don’t skeletons fight each other? Because they don’t have the guts!", 
                      "Why did the scarecrow win an award? Because he was outstanding in his field!", 
                      "I told my wife she should embrace her mistakes. She gave me a hug."]
    },
    {
        "tag": "motivation",
        "patterns": ["Give me some motivation", "I need inspiration", "Motivate me", "Tell me a motivational quote"],
        "responses": ["Success is not final, failure is not fatal: it is the courage to continue that counts. - Winston Churchill", 
                      "Don't watch the clock; do what it does. Keep going. - Sam Levenson", 
                      "Believe in yourself and all that you are. Know that there is something inside you that is greater than any obstacle."]
    },
    {
        "tag": "fun_fact",
        "patterns": ["Tell me a fun fact", "Give me an interesting fact", "Do you know any cool facts?"],
        "responses": ["Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly good to eat!", 
                      "Octopuses have three hearts and their blood is blue!", 
                      "Bananas are berries, but strawberries aren't!"]
    },
    {
        "tag": "history",
        "patterns": ["Tell me a historical fact", "Do you know any history?", "Give me a history lesson"],
        "responses": ["The Great Wall of China is over 13,000 miles long!", 
                      "Did you know that Napoleon Bonaparte was once attacked by a horde of bunnies?", 
                      "The shortest war in history was between Britain and Zanzibar on August 27, 1896. It lasted only 38 to 45 minutes."]
    },
    {
        "tag": "food",
        "patterns": ["What's your favorite food?", "Recommend me something to eat", "What should I cook today?"],
        "responses": ["How about some pasta? It's easy to make and delicious!", 
                      "You can try making a sandwich with fresh veggies and grilled chicken!", 
                      "If you're in the mood for something sweet, how about baking some cookies?"]
    },
    {
        "tag": "books",
        "patterns": ["Can you recommend a book?", "What should I read?", "Suggest a good book"],
        "responses": ["If you love mystery, try 'Sherlock Holmes' by Arthur Conan Doyle.", 
                      "For self-improvement, 'Atomic Habits' by James Clear is a must-read!", 
                      "If you enjoy fantasy, 'Harry Potter' by J.K. Rowling is a great choice."]
    },
    {
        "tag": "movies",
        "patterns": ["Suggest a movie", "What movie should I watch?", "Recommend me a film"],
        "responses": ["If you like action, try 'John Wick'.", 
                      "For a great comedy, watch 'The Hangover'.", 
                      "If you're into sci-fi, 'Interstellar' is a must-watch!"]
    },
    {
        "tag": "music",
        "patterns": ["Recommend me a song", "What song should I listen to?", "Suggest a good song"],
        "responses": ["If you love pop, try 'Blinding Lights' by The Weeknd.", 
                      "For something relaxing, listen to 'Weightless' by Marconi Union.", 
                      "If you're into rock, 'Bohemian Rhapsody' by Queen is a classic."]
    },
    {
        "tag": "time",
        "patterns": ["What time is it?", "Tell me the time", "Can you give me the current time?"],
        "responses": ["I'm not sure, but you can check the clock on your device!", 
                      "Time is an illusion, but your phone should have the answer!", 
                      "I don't have a watch, but you might!"]
    },
    {
        "tag": "sleep",
        "patterns": ["I can't sleep", "Give me some tips for better sleep", "How to fall asleep quickly?"],
        "responses": ["Try listening to calming music or white noise.", 
                      "Avoid screens and bright lights before bed.", 
                      "Read a book or do deep breathing exercises to relax."]
    },
    {
        "tag": "exercise",
        "patterns": ["Suggest a workout", "How can I stay fit?", "Give me some exercise tips"],
        "responses": ["Try a quick 15-minute workout: push-ups, squats, and jumping jacks!", 
                      "A daily 30-minute walk can do wonders for your health!", 
                      "Yoga is great for both the body and mind—give it a try!"]
    }
]

